Step 1: download and convert dataset from CSV to Parquet

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask.distributed import Client, LocalCluster
import dask
from dask.distributed import get_task_stream

In [3]:
import time

In [4]:
print('pandas version: %s' % pd.__version__)
print('numpy version: %s' % np.__version__)
print('dask version: %s' % dask.__version__)

pandas version: 1.3.4
numpy version: 1.20.3
dask version: 2021.10.0


https://docs.databricks.com/_static/notebooks/koalas-benchmark-distributed-execution.html?_ga=2.216403934.95291449.1648935555-599276868.1645477063

In [5]:
filename = "taxi_dataset.txt"

In [6]:
with open(filename) as file:
    csv_files = [line.rstrip() for line in file]
# only choose yellow taxis
yellow = list(filter(lambda x: "yellow" in x, csv_files))

In [7]:
# make the list small for now
yellow = ['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [8]:
yellow

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [9]:
import re
pattern = '/[^\/]*\.csv$/gm'

for csv_url in yellow:
#     get the filename only so I can sort by month / year
    csv_name = re.findall(r"[^\/]*\.csv$",csv_url)
    df = dd.read_csv(csv_url,dtype={'tolls_amount': 'float64'})

    df = df.repartition(npartitions=4)
    df.to_parquet(f'./tmp/trip_data_{csv_name}', write_index=False)


/Users/mk/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/Users/mk/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/Users/mk/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(


Read in parquet to dask

In [47]:
from dask.distributed import get_task_stream

client = Client()

{'description': {'filename': '', 'name': '', 'line_number': 0, 'line': ''},
 'children': {},
 'count': 0,
 'identifier': 'root'}

# set up testbench

In [35]:
def benchmark(f, df, benchmarks, task_name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    ret_benchmark_vals = {}
    with get_task_stream(plot='save', filename="task-stream.html") as ts:
        start_time = time.time()
        ret = f(df, **kwargs)
        ret_benchmark_vals['raw_duration'] = time.time() - start_time
        ret_benchmark_vals['history'] = ts.data
    benchmarks[task_name] = ret_benchmark_vals
    print(f"{task_name} took: {benchmarks[task_name].get('raw_duration')} seconds")
    return benchmarks[task_name].get("raw_duration")

In [31]:
import collections
dask_benchmarks = collections.defaultdict(dict)
# benchmarks = {"task1" : {"stat1": val, "stat2": val}}

# Define benchmark tasks

In [64]:
all_tasks = []

In [65]:
# sum, then means = simple mapreduce
def read_to_basic_ETL(df = None):
    df = dd.read_parquet(
    "./tmp/", 
    storage_options={"anon": True, 'use_ssl': True})    
    return (df.fare_amount + df.tip_amount).mean().compute()
all_tasks.append(read_to_basic_ETL)

In [66]:
# counts of values seen = simple map, groupby, reduce
def count_values(df):
    return df.fare_amount.value_counts().compute()
all_tasks.append(count_values)

In [67]:
# cpu heavy arithmetic : mapreduce
def complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.compute()
all_tasks.append(complicated_arithmetic_operation)

In [68]:
def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg(
      {
        'total_amount': ['mean', 'std'], 
        'tip_amount': ['mean', 'std']
      }
    ).compute()
all_tasks.append(complicated_arithmetic_operation)

In [69]:
# join two datasets
def join_data(df):
    return dd.merge(df, other, left_index=True, right_index=True).compute()
all_tasks.append(complicated_arithmetic_operation)

# run the tasks

In [70]:
df = dd.read_parquet(
    "./tmp/", 
    storage_options={"anon": True, 'use_ssl': True})

In [71]:
# for task in all_tasks:
#     benchmark(task, df=df, benchmarks = dask_benchmarks, task_name = task.__name__)

read_to_basic_ETL took: 0.4521312713623047 seconds
count_values took: 0.34616804122924805 seconds
complicated_arithmetic_operation took: 3.579279661178589 seconds
complicated_arithmetic_operation took: 2.8689677715301514 seconds
complicated_arithmetic_operation took: 2.600781202316284 seconds


In [73]:
[benchmark(task, df=df, benchmarks = dask_benchmarks, task_name = task.__name__) for task in all_tasks]

read_to_basic_ETL took: 0.46721887588500977 seconds
count_values took: 0.24425888061523438 seconds
complicated_arithmetic_operation took: 2.843494176864624 seconds
complicated_arithmetic_operation took: 1.7494018077850342 seconds
complicated_arithmetic_operation took: 1.8757166862487793 seconds


[0.46721887588500977,
 0.24425888061523438,
 2.843494176864624,
 1.7494018077850342,
 1.8757166862487793]

# try to do something fancy with the history

In [114]:
hx = dask_benchmarks.get("read_to_basic_ETL").get("history")

use dask to do dask :D

In [86]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [88]:
hx_df = pd.DataFrame (hx, columns = ['worker','status','nbytes', 'thread', 'type', 'typename', 'metadata', 'startstops', 'key'])

In [97]:
hx_ddf = dd.from_pandas(hx_df, npartitions=1)

In [133]:
# the startstops are nested. we need to unnest this for action stuff only. but use nested for all other
hx_ddf.head(20)

,worker,status,nbytes,thread,type,typename,metadata,startstops,key
0,tcp://127.0.0.1:58544,OK,27191704,123145529507840,b'\x80\x04\x95!\x00\x00\x00\x00\x00\x00\x00\x8c\x12pandas.core.series\x94\x8c\x06Series\x94\x93\x94.',pandas.core.series.Series,{},"({'action': 'compute', 'start': 1649539629.0290596, 'stop': 1649539629.2514455},)","('add-42c91fcd244541e3b459e8913af2d07a', 0)"
1,tcp://127.0.0.1:58544,OK,32,123145529507840,b'\x80\x04\x95\x15\x00\x00\x00\x00\x00\x00\x00\x8c\x05numpy\x94\x8c\x07float64\x94\x93\x94.',numpy.float64,{},"({'action': 'compute', 'start': 1649539629.256208, 'stop': 1649539629.266967},)","('series-sum-chunk-fbd709dae3d9d13e17866642e9f8e505', 0, 0, 0)"
2,tcp://127.0.0.1:58544,OK,32,123145546297344,b'\x80\x04\x95\x13\x00\x00\x00\x00\x00\x00\x00\x8c\x05numpy\x94\x8c\x05int64\x94\x93\x94.',numpy.int64,{},"({'action': 'compute', 'start': 1649539629.2559612, 'stop': 1649539629.268448},)","('series-count-chunk-33f4ebe1295ecdc32b8301290d7befce', 0, 0, 0)"
3,tcp://127.0.0.1:58545,OK,30486864,123145657192448,b'\x80\x04\x95!\x00\x00\x00\x00\x00\x00\x00\x8c\x12pandas.core.series\x94\x8c\x06Series\x94\x93\x94.',pandas.core.series.Series,{},"({'action': 'compute', 'start': 1649539629.02956, 'stop': 1649539629.2818692},)","('add-42c91fcd244541e3b459e8913af2d07a', 1)"
4,tcp://127.0.0.1:58545,OK,32,123145657192448,b'\x80\x04\x95\x15\x00\x00\x00\x00\x00\x00\x00\x8c\x05numpy\x94\x8c\x07float64\x94\x93\x94.',numpy.float64,{},"({'action': 'compute', 'start': 1649539629.285232, 'stop': 1649539629.2956278},)","('series-sum-chunk-fbd709dae3d9d13e17866642e9f8e505', 0, 1, 0)"
5,tcp://127.0.0.1:58545,OK,32,123145640402944,b'\x80\x04\x95\x13\x00\x00\x00\x00\x00\x00\x00\x8c\x05numpy\x94\x8c\x05int64\x94\x93\x94.',numpy.int64,{},"({'action': 'compute', 'start': 1649539629.2851522, 'stop': 1649539629.3018951},)","('series-count-chunk-33f4ebe1295ecdc32b8301290d7befce', 0, 1, 0)"
6,tcp://127.0.0.1:58542,OK,27417264,123145619705856,b'\x80\x04\x95!\x00\x00\x00\x00\x00\x00\x00\x8c\x12pandas.core.series\x94\x8c\x06Series\x94\x93\x94.',pandas.core.series.Series,{},"({'action': 'compute', 'start': 1649539629.0302684, 'stop': 1649539629.3189504},)","('add-42c91fcd244541e3b459e8913af2d07a', 2)"
7,tcp://127.0.0.1:58542,OK,32,123145619705856,b'\x80\x04\x95\x15\x00\x00\x00\x00\x00\x00\x00\x8c\x05numpy\x94\x8c\x07float64\x94\x93\x94.',numpy.float64,{},"({'action': 'compute', 'start': 1649539629.3196013, 'stop': 1649539629.3293502},)","('series-sum-chunk-fbd709dae3d9d13e17866642e9f8e505', 0, 2, 0)"
8,tcp://127.0.0.1:58542,OK,32,123145602916352,b'\x80\x04\x95\x13\x00\x00\x00\x00\x00\x00\x00\x8c\x05numpy\x94\x8c\x05int64\x94\x93\x94.',numpy.int64,{},"({'action': 'compute', 'start': 1649539629.3195152, 'stop': 1649539629.335808},)","('series-count-chunk-33f4ebe1295ecdc32b8301290d7befce', 0, 2, 0)"
9,tcp://127.0.0.1:58543,OK,30016288,123145528463360,b'\x80\x04\x95!\x00\x00\x00\x00\x00\x00\x00\x8c\x12pandas.core.series\x94\x8c\x06Series\x94\x93\x94.',pandas.core.series.Series,{},"({'action': 'compute', 'start': 1649539629.0295646, 'stop': 1649539629.3822947},)","('add-42c91fcd244541e3b459e8913af2d07a', 3)"


In [188]:
# the startstops are nested. we need to unnest this for action stuff only.
exploded_df = hx_ddf.explode("startstops")

In [226]:
# AAAAGH THIS TOOK FOREVER TO FIGURE OUT O_O
exploded_df['action'] = exploded_df['startstops'].apply(lambda x: x['action'], meta = ("action", str))
exploded_df['start'] = exploded_df['startstops'].apply(lambda x: x['start'], meta = ("start", np.float64))
exploded_df['end'] = exploded_df['startstops'].apply(lambda x: x['stop'], meta = ("stop", np.float64))
exploded_df['action_duration'] = exploded_df['end'] - exploded_df['start']

In [247]:
exploded_df_only_agg_fields = exploded_df[['worker', 'action', 'action_duration']]

In [251]:
time_per_worker_and_action = exploded_df_only_agg_fields.groupby(['worker','action']).agg("sum")

This is final for time_per_worker_and_action

In [252]:
time_per_worker_and_action.head(20)

action_duration
worker                action                   
tcp://127.0.0.1:58542 compute          0.314724
tcp://127.0.0.1:58543 compute          0.377976
                      transfer         0.005016
tcp://127.0.0.1:58544 compute          0.245632
tcp://127.0.0.1:58545 compute          0.280231
                      transfer         0.006547

In [ ]:
# now get nbytes per worker

In [255]:
nbytes_per_worker = hx_ddf[['worker', 'nbytes']].groupby(["worker"]).agg("sum")

In [256]:
nbytes_per_worker.head(20)

,nbytes
worker,
tcp://127.0.0.1:58542,27417328
tcp://127.0.0.1:58543,30016384
tcp://127.0.0.1:58544,27191768
tcp://127.0.0.1:58545,30486992
